# Diffusion Based Models (2022)

## Idea

**Main Idea**

Diffusion Based models are a type of generative models that create high-quality data by gradually refining noise into structured patterns, typically using a process that reverses diffusion, which is the gradual mixing of substances

**Pros**

1. **Detail Preservation:** Effective at maintaining details in generated images or patterns
2. **Control:** Allow more control over the generation process through conditioning
3. **Stability:** Tend to be more stable during training compared to traditional GANs

**Cons**

- **Complexity:** The underlying mechanisms can be complex and computationally intensive
- **Hyperparameter Sensitivity:** May require careful tuning of hyperparameters for optimal performance

**Creator**

Stable Diffusion — модель генерации изображений, созданная при сотрудничестве Runway ML и CompViz group в LMU Munich с использованием ресурсов Stability AI

- https://github.com/Stability-AI/generative-models

![https://www.linkedin.com/pulse/diffusion-model-generative-image-synthesis-yogeshwaran-singarasu-jgo4c/](content/img4001_SDidea.png "SDidea")

### Forward Diffusion Process


- A controlled, fixed process
- Self-determined

![https://theaisummer.com/diffusion-models/](content/img4002_Forward.png "Forward")

$$ q(x_t | x_{t-1}) = \mathcal{N}(x_t; \mu_t = \sqrt{1 - \beta_t} x_{t-1}, \Sigma_t = \beta_t I) $$

- $x_t$ — state of the image at time step $t$
- $x_{t-1}$ — state of the image at the previous step $t-1$
- $\beta_t$ — diffusion coefficient at step $t$
- $\mu_t$ —  mean of the normal distribution
- $\Sigma_t$ —  covariance matrix of the normal distribution, where $I$ represents the identity matrix

![https://yang-song.net/blog/2021/score/](content/img4003_FNoising.gif "FNoising")

- $x_t = \sqrt{\alpha_t} x_{t-1} + \sqrt{1 - \alpha_t} \varepsilon_t, \quad \varepsilon_t \sim \mathcal{N}(0, I), \quad \alpha_t = 1 - \beta_t $
- $x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 - \bar{\alpha}_t} \varepsilon, \quad \varepsilon \sim \mathcal{N}(0, I), \quad \bar{\alpha}_t = \prod_{i=1}^t \alpha_i $
- $x_{t-1} = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \varepsilon \right) + \tilde{\beta}_t z, \quad z \sim \mathcal{N}(0, I), \quad \tilde{\beta}_t = \frac{1 - \bar{\alpha}_{t-1}}{1 - \bar{\alpha}_t} \beta_t$
- $\varepsilon$ - noise vector



### Reverse Diffusion Process

- Trained by the model

![https://theaisummer.com/diffusion-models/](content/img4004_Reverse.png "Reverse")

![https://yang-song.net/blog/2021/score/](content/img4005_RNoising.gif "RNoising")

- $\hat{x}_{t-1} = \frac{1}{\sqrt{\bar{\alpha}_t}} \left( x_t - \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}} \varepsilon_{\theta}(x_t, t) \right) + \sigma_t z, \quad z \sim \mathcal{N}(0, I), \quad \sigma_t = \text{const}$
    - Here $\hat{\mathbf{x}}_{t-1}$ - predicted image at step $t-1$
    - $\varepsilon_{\theta}$ is the vector of predicted noise
    - $\sigma_t z$ - is a regularization element, for better training of the network

**Loss (MSE)**
- $L_t = \|\mathbf{x}_{t-1} - \hat{\mathbf{x}}_{t-1}\|_2^2 \propto \|\boldsymbol{\varepsilon} - \boldsymbol{\varepsilon}_{\theta}(\mathbf{x}_t, t)\|_2^2 \rightarrow \min_{\theta}$
    - $\boldsymbol{\varepsilon}_{\theta}(\mathbf{x}_t, t)\$ - image, time 

![https://arxiv.org/pdf/2006.11239.pdf](content/img4006_Algo.png "Algo")

We predict epsilon for the image and predict the conditional distribution, given that we already know the image \(X_t\). \
It can't be done analytically, i.e., predicting the parameters of this distribution (for example, using the maximum likelihood method) is difficult because:

- The distribution is conditional.
- It has high dimensionality.

Therefore, neural networks are used—they solve the likelihood optimization prb\
lem. Moreover, we need to do this multiple times, calculating the conditional distribution, the conditionally conditional distribution etc.

**Hyperparameters**

- $ T = 1000 $
- $ \beta_1 < \beta_2 < \cdots < \beta_t < \cdots < \beta_T $
- $ \beta_1 = 0.0001, \beta_T = 0.02 $
- $ \sigma_t^2 = \beta_t \quad \text {or} \quad \sigma_t^2 = \tilde{\beta}_t $

**Conditional Diffusion Models (with text)**

**Loss (MSE)**
- $L_t = \|\mathbf{x}_{t-1} - \hat{\mathbf{x}}_{t-1}\|_2^2 \propto \|\boldsymbol{\varepsilon} - \boldsymbol{\varepsilon}_{\theta}(\mathbf{x}_t, t)\|_2^2 \rightarrow \min_{\theta}$
    - $\boldsymbol{\varepsilon}_{\theta}(\mathbf{x}_t, t)\$ - картинка, время, условие (эмбеддинг из трансформера из текста)

## Stable Diffusion Implementation

![https://generativeai.pub/denoising-diffusion-probabilistic-models-from-scratch-728df8228565](content/img4007_Imp1.webp "Imp1")

![https://medium.com/augmented-startups/stable-diffusion-the-engine-powering-ai-influencer-stardom-ee64b4f76101](content/img4008_Imp2.png "Imp2")

**Основные блоки**
- **Encoder (E)** – превращает изображение из пиксельного пространства в латентный вектор (сжатое представление)
- **Latent Space** – здесь происходит диффузия: добавляется шум к латенту (прямой процесс), затем денойзинг восстанавливает его (обратный процесс)
- **Denoising U-Net** – denoising UNet. Она состоит из downsample блоков, mid-блока и upsample блоков. \
  Каждый из таких блоков является смесью нескольких последовательных resnet и cross-attention блоков. \
  Последние используются, чтобы модель начала реагировать на какой-то внешний сигнал, например, на эмбеддинги текстового промпта.
- **Conditioning / Transformer (Tθ)** – добавляет условие (текст, семантическая карта, другое изображение) для управления генерацией
- **Decoder (D)** – превращает очищенный латент обратно в сгенерированное изображение.

**Версии моделей**
1. **Stable Diffusion v1.1 - v1.5** ([CompViz репозиторий](https://github.com/CompVis/stable-diffusion), [RunwayML репозиторий](https://github.com/runwayml/stable-diffusion)). Обучение проходило на подвыборке из [LAION-Aesthetics V2](https://laion.ai/blog/laion-aesthetics/#laion-aesthetics-v2) в разрешении 512x512. Модель использует подход с $\epsilon$-предсказанием.
2. **Stable Diffusion v2.0 - v2.1** ([StabilityAI репозиторий](https://github.com/Stability-AI/stablediffusion)). Обучение осуществлялось на подвыборке [LAION-5B](https://laion.ai/blog/laion-5b/) в разрешении 768x768. Модель применяет подход с $v$-предсказанием. Она становится больше, обучается с нуля и использует более тяжелую версию [OpenCLIP](https://github.com/mlfoundations/open_clip) вместо OpenAI CLIP.
3. **Stable Diffusion XL** ([StabilityAI репозиторий](https://github.com/Stability-AI/generative-models), [arXiv:2307.01952](https://arxiv.org/abs/2307.01952)). Модель имеет другую архитектуру и работает в две стадии: base и refiner. Включает в себя целых 2 CLIP text encoder’а. При обучении использовались дополнительные трюки — Micro-Conditioning и [SDEdit](https://arxiv.org/abs/2108.01073). Обучение проходило в разрешении 1024x1024. Модель снова стала применять подход с $\epsilon$-предсказанием. 
4. **Stable Diffusion 3** ([блогпост](https://stability.ai/news/stable-diffusion-3), [arxiv.org:2403.03206](https://arxiv.org/pdf/2403.03206)). Сейчас модель не находится в публичном доступе, но доступна по API и показывает потрясающие результаты. Из основных изменений: заметно изменена архитектура модели (используется DiT вместо Unet и 3 text encoder’а, в процессе инференса сам text condition обрабатывается вместе с латентами параллельно, обучение базируется на концепции Rectified Flo
5. **Stable Diffusion XL Turbo** ([StabilityAI HF](https://huggingface.co/stabilityai/sdxl-turbo)) - Упрощённая и ускоренная версия SDXL, работает за 1–4 шага генерации. Использует подход Adversarial Diffusion Distillation (ADD) и ориентирована на реалтайм

---
**Попробуйте реализовать Diffusion модель сами: https://habr.com/ru/articles/860400/**

### [**AUTOMATIC1111 / Stable Diffusion web UI**](https://github.com/AUTOMATIC1111/stable-diffusion-webui)

![https://deepschool-pro.notion.site/Stable-Diffusion-961e8a9430a64119ad76999652c658c3](content/img4009_AUTOMATIC1111.jpg "AUTOMATIC1111")

### [**ComfyUI**](https://github.com/comfyanonymous/ComfyUI)

![https://deepschool-pro.notion.site/Stable-Diffusion-961e8a9430a64119ad76999652c658c3](content/img4010_ComfyUI.jpg "ComfyUI")

### [**Diffusers**](https://huggingface.co/docs/diffusers/en/index)

## Projects

# Links

- https://www.youtube.com/watch?v=z7QH-WWHmfE
- https://theaisummer.com/diffusion-models/
- https://deepschool-pro.notion.site/Stable-Diffusion-961e8a9430a64119ad76999652c658c3

**Evaluation**

- https://huggingface.co/docs/diffusers/en/conceptual/evaluation

# END